In [2]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf

In [3]:
# Import wheat pricing data
wheat_prices = pd.read_excel('../Cleaned Data/Wheat_Prices_clean.xlsx', sheet_name = 'Table18')
wheat_prices = wheat_prices[wheat_prices['Class'] == 'All wheat']

#convert to long form
wheat_prices = pd.melt(
    wheat_prices,
    id_vars=['Marketing year 1/'], 
    value_vars=['Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May'],
    var_name='Month',
    value_name='wheat_price'
)

#drop NaN
wheat_prices = wheat_prices[wheat_prices['wheat_price'] != '--']

#Save year
wheat_prices['Year'] = wheat_prices['Marketing year 1/'].str.split('/').str[0]
second_year = wheat_prices['Month'].isin(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun'])
wheat_prices.loc[second_year, 'Year'] = wheat_prices.loc[second_year, 'Year'].astype(int) + 1
wheat_prices['Year'] = wheat_prices['Year'].astype(str)

#order by month
wheat_prices = wheat_prices.drop(columns = 'Marketing year 1/')
month_order = {'Jul': 1, 'Aug': 2, 'Sep': 3, 'Oct': 4, 'Nov': 5, 'Dec': 6, 
               'Jan': 7, 'Feb': 8, 'Mar': 9, 'Apr': 10, 'May': 11, 'Jun': 12}
wheat_prices['Month_num'] = wheat_prices['Month'].map(month_order)
wheat_prices = wheat_prices.sort_values(['Year', 'Month_num'])
wheat_prices = wheat_prices.drop('Month_num', axis=1).reset_index()
wheat_prices = wheat_prices[['Year', 'Month', 'wheat_price']].reset_index(drop = True)
wheat_prices['Wheat_Price'] = pd.to_numeric(wheat_prices['wheat_price'], errors='coerce').dropna()
wheat_prices.head()

,Year,Month,wheat_price,Wheat_Price
0,1908,Jul,0.9,0.900
1,1908,Aug,0.896,0.896
2,1908,Sep,0.896,0.896
3,1908,Oct,0.91,0.910
4,1908,Nov,0.922,0.922


In [4]:
# Import fertilizer data
fertilizer_index = pd.read_csv('../Cleaned Data/Fertilizer_Index_Data_clean.csv')

#save year and month from observation_date
fertilizer_index['Year'] = pd.to_datetime(fertilizer_index['observation_date']).dt.year
fertilizer_index['Month'] = pd.to_datetime(fertilizer_index['observation_date']).dt.strftime('%b')

# Rename and reorder
fertilizer_index = fertilizer_index.rename(columns={'PCU325311325311': 'fertilizer_idx'})
fertilizer_index = fertilizer_index[['Year', 'Month', 'fertilizer_idx']]
fertilizer_index.head()

,Year,Month,fertilizer_idx
0,1975,Dec,88.1
1,1976,Jan,87.6
2,1976,Feb,85.0
3,1976,Mar,84.8
4,1976,Apr,84.7


In [5]:
# Import temperature data
temp_data = pd.read_csv('../Cleaned Data/Temperature_Data_clean.csv')

#save just USA
temp_data = temp_data[temp_data['Code'] == 'USA']

#convert to long form
year_columns = [col for col in temp_data.columns if str(col).isdigit()]
temp_data = pd.melt(
    temp_data, 
    id_vars=['Entity', 'Code', 'Year'], 
    value_vars=year_columns,
    var_name='Year_col', 
    value_name='value'
)

#change how month is shown
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 
             7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
temp_data['Month'] = temp_data['Year'].map(month_map)


temp_data = temp_data.drop(columns = ['Entity', 'Code', 'Year'])
temp_data = temp_data.rename(columns={'Year_col': 'Year'})

#Sort by Year and Month
month_order = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
               'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
temp_data['Month_num'] = temp_data['Month'].map(month_order)
temp_data = temp_data.sort_values(['Year', 'Month_num'])
temp_data = temp_data.drop('Month_num', axis=1)

#drop empty values
temp_data = temp_data.dropna(subset=['value']).reset_index(drop=True)
temp_data = temp_data.rename(columns = {'value': 'avg_temp'}) #degrees C
temp_data = temp_data[['Year', 'Month', 'avg_temp']]
temp_data.head()

,Year,Month,avg_temp
0,1950,Jan,-3.167346
1,1950,Feb,-2.178927
2,1950,Mar,1.562352
3,1950,Apr,6.770633
4,1950,May,13.099970


In [6]:
# Import monthly cpi data
cpi_data = pd.read_excel('../Cleaned Data/cpi_data_clean.xlsx', sheet_name = 'Monthly')

#convert to datetime; save year and month
cpi_data['observation_date'] = pd.to_datetime(cpi_data['observation_date'])
cpi_data['Year'] = cpi_data['observation_date'].dt.year
cpi_data['Month'] = cpi_data['observation_date'].dt.strftime('%b')

#rename col
cpi_data = cpi_data.rename(columns={'CPIAUCSL': 'cpi'})
cpi_data = cpi_data[['Year', 'Month', 'cpi']].reset_index(drop = True)
cpi_data.head()

,Year,Month,cpi
0,1947,Jan,21.48
1,1947,Feb,21.62
2,1947,Mar,22.00
3,1947,Apr,22.00
4,1947,May,21.95


In [7]:
# Import US import data
import_data = pd.read_excel('../Cleaned Data/Commodity_Import_Index_clean.xlsx', sheet_name = 'Monthly')

#convert to datetime; save year and month
import_data['observation_date'] = pd.to_datetime(import_data['observation_date'])
import_data['Year'] = import_data['observation_date'].dt.year
import_data['Month'] = import_data['observation_date'].dt.strftime('%b')

#rename
import_data = import_data.rename(columns={'IR': 'commodity_imp_idx'})

#aggregate by average
import_data = import_data.groupby(['Year', 'Month']).agg({'commodity_imp_idx': 'mean'}).reset_index()
import_data = import_data[['Year', 'Month', 'commodity_imp_idx']].reset_index(drop = True)

#save only years with obs for each month
import_data = import_data[import_data['Year'] > 1988]

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
import_data['Month'] = pd.Categorical(import_data['Month'], categories=month_order, ordered=True)
import_data = import_data.sort_values(['Year', 'Month'])

import_data.head()

,Year,Month,commodity_imp_idx
80,1989,Jan,91.1
79,1989,Feb,90.6
83,1989,Mar,91.3
76,1989,Apr,92.0
84,1989,May,92.6


In [8]:
# Import US export data
export_data = pd.read_excel('../Cleaned Data/Commodity_Export_Index_Data_clean.xlsx', sheet_name = 'Monthly')

#convert to datetime; save year and month
export_data['observation_date'] = pd.to_datetime(export_data['observation_date'])
export_data['Year'] = export_data['observation_date'].dt.year
export_data['Month'] = export_data['observation_date'].dt.strftime('%b')

#rename
export_data = export_data.rename(columns={'IQ': 'commodity_exp_idx'})

#aggregate by avg
export_data = export_data.groupby(['Year', 'Month']).agg({'commodity_exp_idx': 'mean'}).reset_index()
export_data = export_data[['Year', 'Month', 'commodity_exp_idx']].reset_index(drop = True)

#save only years with obs for each month
export_data = export_data[export_data['Year'] > 1988]

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
export_data['Month'] = pd.Categorical(export_data['Month'], categories=month_order, ordered=True)
export_data = export_data.sort_values(['Year', 'Month'])

export_data.head()

,Year,Month,commodity_exp_idx
68,1989,Jan,94.9
67,1989,Feb,94.6
71,1989,Mar,95.2
64,1989,Apr,95.0
72,1989,May,95.4


In [9]:
# Import Oil Price Data
crude_oil_price = pd.read_excel('../Cleaned Data/Crude_Oil_Prices_clean.xlsx', sheet_name = 'Monthly')

#convert to datetime; save year and month
crude_oil_price['observation_date'] = pd.to_datetime(crude_oil_price['observation_date'])
crude_oil_price['Year'] = crude_oil_price['observation_date'].dt.year
crude_oil_price['Month'] = crude_oil_price['observation_date'].dt.strftime('%b')

#rename
crude_oil_price = crude_oil_price.rename(columns={'MCOILWTICO': 'crude_oil_price'})

#aggregate by avg
crude_oil_price = crude_oil_price.groupby(['Year', 'Month']).agg({'crude_oil_price': 'mean'}).reset_index()
crude_oil_price = crude_oil_price[['Year', 'Month', 'crude_oil_price']].reset_index(drop = True)

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
crude_oil_price['Month'] = pd.Categorical(crude_oil_price['Month'], categories=month_order, ordered=True)
crude_oil_price = crude_oil_price.sort_values(['Year', 'Month'])

crude_oil_price.head()

,Year,Month,crude_oil_price
4,1986,Jan,22.93
3,1986,Feb,15.46
7,1986,Mar,12.61
0,1986,Apr,12.84
8,1986,May,15.38


In [10]:
# Import Drought index data 
drought_index = pd.read_csv('../Cleaned Data/Drought_Index_Data_clean.csv')
drought_index = drought_index[['DATE', 'D2']].copy() #Note: Using D2 values since this is most linked to agricultural areas in the US

#change date to datetime
drought_index['DATE'] = pd.to_datetime(
    drought_index['DATE'].str.replace('d_', ''), format='%Y%m%d'
)

#save month and year
drought_index['Year'] = drought_index['DATE'].dt.year
drought_index['Month'] = drought_index['DATE'].dt.strftime('%b')

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
drought_index['Month'] = pd.Categorical(drought_index['Month'], categories=month_order, ordered=True)
drought_index = drought_index.sort_values(['Year', 'Month']).dropna()

#aggregate by mean for each month
drought_index = drought_index.groupby(['Year','Month']).agg({'D2': 'mean'}).reset_index()
drought_index = drought_index.rename(columns = {'D2': 'drought_idx'}).dropna()
drought_index

/var/folders/dw/zsq0tshd17zfndgp1s_yw9zc0000gn/T/ipykernel_84093/2538829274.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  drought_index = drought_index.groupby(['Year','Month']).agg({'D2': 'mean'}).reset_index()


,Year,Month,drought_idx
0,1895,Jan,0.0
1,1895,Feb,0.0
2,1895,Mar,0.0
3,1895,Apr,0.0
4,1895,May,0.0
...,...,...,...
1558,2024,Nov,5.7
1559,2024,Dec,7.1
1560,2025,Jan,12.0
1561,2025,Feb,15.9


In [11]:
# Import flour price data
flour_price = pd.read_excel('../Cleaned Data/Flour_Price_data_clean.xlsx', sheet_name = 'Monthly')

#convert to datetime; save year and month
flour_price['observation_date'] = pd.to_datetime(flour_price['observation_date'])
flour_price['Year'] = flour_price['observation_date'].dt.year
flour_price['Month'] = flour_price['observation_date'].dt.strftime('%b')

#rename
flour_price = flour_price.rename(columns={'APU0000701111': 'flour_price'})

#save last value of the month
flour_price = flour_price.groupby(['Year', 'Month']).last().reset_index()
flour_price = flour_price[['Year', 'Month', 'flour_price']].reset_index(drop = True)

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
flour_price['Month'] = pd.Categorical(flour_price['Month'], categories=month_order, ordered=True)
flour_price = flour_price.sort_values(['Year', 'Month'])

flour_price.head()

,Year,Month,flour_price
4,1980,Jan,0.203
3,1980,Feb,0.205
7,1980,Mar,0.211
0,1980,Apr,0.206
8,1980,May,0.207


In [12]:
# Import corn price data (substitute)
corn_price = pd.read_csv('../Cleaned Data/Corn_Price_Data_clean.csv')

#convert to datetime; save year and month
corn_price['observation_date'] = pd.to_datetime(corn_price['observation_date'])
corn_price['Year'] = corn_price['observation_date'].dt.year
corn_price['Month'] = corn_price['observation_date'].dt.strftime('%b')

#rename
corn_price = corn_price.rename(columns={'PMAIZMTUSDM': 'corn_price'})

#save last value of the month
corn_price = corn_price.groupby(['Year', 'Month']).last().reset_index()
corn_price = corn_price[['Year', 'Month', 'corn_price']].reset_index(drop = True)

#sort jan-dec
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
corn_price['Month'] = pd.Categorical(corn_price['Month'], categories=month_order, ordered=True)
corn_price = corn_price.sort_values(['Year', 'Month'])

corn_price.head()

,Year,Month,corn_price
4,1990,Jan,105.900513
3,1990,Feb,106.294167
7,1990,Mar,109.837318
0,1990,Apr,118.498337
8,1990,May,121.254097


In [13]:
# Convert Year columns to string in all dataframes before merging
wheat_prices['Year'] = wheat_prices['Year'].astype(str)
fertilizer_index['Year'] = fertilizer_index['Year'].astype(str)
temp_data['Year'] = temp_data['Year'].astype(str)
cpi_data['Year'] = cpi_data['Year'].astype(str)
import_data['Year'] = import_data['Year'].astype(str)
export_data['Year'] = export_data['Year'].astype(str)
crude_oil_price['Year'] = crude_oil_price['Year'].astype(str)
drought_index['Year'] = drought_index['Year'].astype(str)
flour_price['Year'] = flour_price['Year'].astype(str)
corn_price['Year'] = corn_price['Year'].astype(str)



#merge datasets
final_data = wheat_prices.copy()
final_data = final_data.merge(fertilizer_index, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(temp_data, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(cpi_data, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(import_data, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(export_data, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(crude_oil_price, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(drought_index, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(flour_price, on = ['Year', 'Month'], how = 'left')
final_data = final_data.merge(corn_price, on = ['Year', 'Month'], how = 'left')
final_data.dropna(inplace = True)

In [14]:
#Saving Month-Year combination as a datetime variable
month_map = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
final_data['Month_Num'] = final_data['Month'].map(month_map)

# Create date variable
final_data['date'] = pd.to_datetime(final_data['Year'].astype(str) + '-' + final_data['Month_Num'].astype(str) + '-01')
final_data = final_data.drop(columns = ['Year', 'Month','Month_Num'])
final_data = final_data.sort_values(by='date')
final_data = final_data[['date','wheat_price','fertilizer_idx',
                         'avg_temp', 'drought_idx','cpi','commodity_imp_idx',
                        'commodity_exp_idx', 'flour_price', 'corn_price',
                        'crude_oil_price']].reset_index(drop=True)
final_data.head()

,date,wheat_price,fertilizer_idx,avg_temp,drought_idx,cpi,commodity_imp_idx,commodity_exp_idx,flour_price,corn_price,crude_oil_price
0,1990-01-01,3.71,114.4,-1.218474,15.3,127.5,92.5,95.2,0.253,105.900513,22.86
1,1990-02-01,3.56,120.1,-1.965352,14.6,128.0,92.7,94.6,0.249,106.294167,22.11
2,1990-03-01,3.48,119.6,4.222106,11.2,128.6,92.4,94.8,0.252,109.837318,20.39
3,1990-04-01,3.49,119.1,9.391660,8.0,128.9,91.6,95.1,0.251,118.498337,18.43
4,1990-05-01,3.4,115.8,13.738063,7.1,129.1,91.2,95.3,0.254,121.254097,18.20


In [15]:
final_data.to_csv('../Final Data/Final_Data.csv', index = False)

In [16]:
wheat_prices.to_csv('../Final Data/Wheat_Time_Series.csv', index = False)